In [6]:
import pandas as pd

# Thêm tham số on_bad_lines='skip' để bỏ qua các dòng bị lỗi cú pháp
# và engine='python' vì nó linh hoạt hơn engine C mặc định trong việc xử lý lỗi
df = pd.read_csv('final.csv', sep=',', on_bad_lines='skip', engine='python')

# Hoặc: on_bad_lines='warn' để giữ lại dòng lỗi nhưng cảnh báo (chỉ dùng nếu bạn cần kiểm tra dòng đó)
# df = pd.read_csv('final.csv', sep=',', on_bad_lines='warn', engine='python')

print("--- 5 dòng đầu tiên sau khi xử lý lỗi ---")
print(df.head())

--- 5 dòng đầu tiên sau khi xử lý lỗi ---
               name                                            comment
0         Sown Hong        Ko dizz lại đc thì cứ ko biết cho nhanh :))
1           Đỗ Việt  Đang bận chạy show kiếm vài tỏi/tháng. Dính dô...
2  Nguyễn Thanh Lan  Tính ra nghe rap nay gần 20 năm t ko biết rich...
3        Tuấn Khanh                     Karik và cục kít của a ấy hả 🤣
4    Nguyễn Tấn Kha  Thú thật Bray không rep thằng Choi thì hầu hết...


In [7]:
missing_values_count = df.isnull().sum()
print("\n--- Số lượng giá trị thiếu ---")
print(missing_values_count)


--- Số lượng giá trị thiếu ---
name       1
comment    8
dtype: int64


In [8]:
# 1. Xóa các hàng có 'comment' bị thiếu (bắt buộc cho việc dán nhãn)
print(f"Số hàng ban đầu: {len(df)}")
df.dropna(subset=['comment'], inplace=True)
print(f"Số hàng sau khi xóa comment thiếu: {len(df)}")

# 2. Điền giá trị cho các hàng 'name' bị thiếu
df['name'].fillna('Vô danh', inplace=True)

# Kiểm tra lại
print("\n--- Kiểm tra lại số lượng giá trị thiếu sau khi xử lý ---")
print(df.isnull().sum())

Số hàng ban đầu: 7515
Số hàng sau khi xóa comment thiếu: 7507

--- Kiểm tra lại số lượng giá trị thiếu sau khi xử lý ---
name       0
comment    0
dtype: int64


C:\Users\hongk\AppData\Local\Temp\ipykernel_17624\3470935241.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['name'].fillna('Vô danh', inplace=True)


In [9]:
# Giả sử bạn đã tải dữ liệu vào DataFrame tên là df
# df = pd.read_csv('final.csv')

# 1. Đếm số lượng bản ghi trùng lặp trong cột 'comment'
duplicate_count = df.duplicated(subset=['comment']).sum()

print(f"Tổng số bình luận trùng lặp (không tính lần xuất hiện đầu tiên): {duplicate_count}")
print(f"Tổng số bình luận duy nhất: {len(df) - duplicate_count}")

# 2. Xem các bản ghi trùng lặp (ví dụ: xem 5 bản trùng lặp đầu tiên)
if duplicate_count > 0:
    duplicate_rows = df[df.duplicated(subset=['comment'], keep=False)].sort_values(by='comment')
    print("\n--- Ví dụ 5 bản ghi bình luận trùng lặp ---")
    print(duplicate_rows.head(10)) # Hiển thị 10 dòng (5 cặp trùng lặp)

Tổng số bình luận trùng lặp (không tính lần xuất hiện đầu tiên): 809
Tổng số bình luận duy nhất: 6698

--- Ví dụ 5 bản ghi bình luận trùng lặp ---
                 name                                            comment
87         Đạt Nguyễn                                                  -
435        Đạt Nguyễn                                                  -
1690       Đạt Nguyễn                                                  -
261        Đạt Nguyễn                                                  -
2010  Nguyễn Minh Tần  1 đất nước cấm học và làm theo những gì mình t...
2166  Nguyễn Minh Tần  1 đất nước cấm học và làm theo những gì mình t...
2088  Nguyễn Minh Tần  1 đất nước cấm học và làm theo những gì mình t...
292    Tran Duc Thanh                   1. Bắt hơn 100 tr dân làm con nợ
466    Tran Duc Thanh                   1. Bắt hơn 100 tr dân làm con nợ
118    Tran Duc Thanh                   1. Bắt hơn 100 tr dân làm con nợ


In [10]:
df_cleaned = df.drop_duplicates(subset=['comment'], keep='first')

print(f"DataFrame đã sẵn sàng để dán nhãn: {len(df_cleaned)} hàng.")

DataFrame đã sẵn sàng để dán nhãn: 6698 hàng.


In [17]:
# Xác định và loại bỏ các bình luận "trống" hoặc "không ý nghĩa"
# Ví dụ: loại bỏ các bình luận chỉ chứa 1 ký tự, hoặc chỉ chứa dấu gạch ngang
comments_to_remove = ['-', '.', '...', '']
df_cleaned = df_cleaned[~df_cleaned['comment'].isin(comments_to_remove)]

# Bạn cũng nên loại bỏ các bình luận có độ dài (số từ) quá ngắn (ví dụ < 2 từ)
df_cleaned['word_count'] = df_cleaned['comment'].apply(lambda x: len(str(x).split()))
df_final = df_cleaned[df_cleaned['word_count'] >= 2].drop(columns=['word_count'])

print(f"DataFrame cuối cùng sau khi loại bỏ bình luận trống: {len(df_final)} hàng.")

DataFrame cuối cùng sau khi loại bỏ bình luận trống: 6558 hàng.


In [ ]:
import pandas as pd
import csv # Cần thiết để sử dụng các hằng số của csv

# Giả sử df_final là DataFrame của bạn
# và 'ten_cot_binh_luan' là tên cột chứa bình luận của bạn

ten_cot_binh_luan = 'Tên cột bình luận' # <<< BẠN HÃY THAY BẰNG TÊN CỘT ĐÚNG

# --- BƯỚC 1: DỌN DẸP MẠNH MẼ HƠN ---
# Xóa bỏ TẤT CẢ các dấu ngoặc kép (") VÀ khoảng trắng ( ) thừa ở hai đầu chuỗi.
# Ví dụ: ' ""Nội dung"" ' sẽ trở thành 'Nội dung'
df_final['comment'] = df_final['comment'].astype(str).str.strip(' "')


# --- BƯỚC 2: XUẤT FILE VỚI QUOTE_ALL ---
# Thay vì QUOTE_MINIMAL (dòng có dòng không), chúng ta dùng QUOTE_ALL
# để buộc pandas bọc ngoặc kép cho TẤT CẢ các trường.
# Điều này đảm bảo tính nhất quán.
df_final.to_csv('df_final_da_lam_sach.csv',
                index=False,
                encoding='utf-8',
                quoting=csv.QUOTE_ALL) # Thay đổi quan trọng ở đây!

print("Đã xuất file thành công: 'df_final_da_lam_sach.csv'")
print("Tất cả các trường (user và comment) bây giờ sẽ được bọc trong ngoặc kép.")

KeyError: 'Tên cột bình luận'